# **ODOC Scraper**

### Library Install

In [2]:
import requests
import os
from datetime import datetime
from datetime import date
from zipfile import *
from zipfile import ZipFile 
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import re
import pickle
import bz2
import _pickle as cPickle

### Functions

In [3]:
# Pickle a file and then compress it into a file with extension 
def compressed_pickle(title, data):
 with bz2.BZ2File(title + '.pbz2', 'w') as f: 
     cPickle.dump(data, f)

    
# Load any compressed pickle file
def decompress_pickle(file):
 data = bz2.BZ2File(file, 'rb')
 data = cPickle.load(data)
 return data

### Download data and place in a zip folder

In [4]:
# zip url
url = 'http://publicdata.doc.state.ok.us/vendor%20extract.zip'

# download the file contents in binary format
r = requests.get(url)

# dd/mm/YY H:M:S
location = "../Paul/ODOC_Raw"
extension = ".zip"
file_name =  location + extension
file_name

# write to zip folder
with open(file_name, "wb") as zip:
    zip.write(r.content)

### Unzip the folder

In [5]:
with ZipFile('ODOC_Raw.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('ODOC_Raw')
   print('File is unzipped in ODOC_Raw folder') 

File is unzipped in ODOC_Raw folder


### Alias Data

In [6]:
# open the alias file and remove whitespace characters like `\n` at the end of each line 
with open("ODOC_Raw/Vendor_Alias_Extract_Text.dat") as f:
    content = f.readlines()
content = [x.strip() for x in content] 

# convert to npy array
alias_arr = np.array(content)

# convert to df
alias_df = pd.DataFrame(alias_arr)

# rename the 1st column
alias_df.rename(columns={ alias_df.columns[0]: "Raw" }, inplace = True)

# PARSE OUT COLUMNS
alias_df['DOC_NUM'] = alias_df['Raw'].str.slice(stop=10)
alias_df['Raw'] = alias_df['Raw'].str.slice(start=10)
alias_df['LAST_NAME'] = alias_df['Raw'].str.slice(stop=30)
alias_df['Raw'] = alias_df['Raw'].str.slice(start=30)
alias_df['FIRST_NAME'] = alias_df['Raw'].str.slice(stop=30)
alias_df['Raw'] = alias_df['Raw'].str.slice(start=30)
alias_df['MIDDLE_NAME'] = alias_df['Raw'].str.slice(stop=30)
alias_df['Raw'] = alias_df['Raw'].str.slice(start=30)
alias_df['SUFFIX'] = alias_df['Raw'].str.slice(stop=5)
alias_df['Raw'] = alias_df['Raw'].str.slice(start=5)

# drop the last row since it is designated to DO NOT USE
alias_df = alias_df[alias_df.MIDDLE_NAME != 'DO NOT USE']

# drop the original 'raw' column
del alias_df['Raw']
alias_df

# write to compressed pickle
compressed_pickle('alias_compressed.pickle', alias_df.to_pickle('alias.pickle'))

alias_df

,DOC_NUM,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX
0,0000008359,MESLES,ARDELL,,
1,0000010337,SAWYER,FRANK,N,
2,0000012556,GOFORTH,PAMELA,B,
3,0000020413,KNIGHTON,RAYMOND,J,
4,0000022574,HOGNER,CLARENCE,,
...,...,...,...,...,...
598879,0000993890,GUINN,JEANETTE,,
598880,0000996281,PILON,GARY,V,
598881,0000997494,AKERS,CONV,,
598882,0000997494,AKERS,MARK,W,


### Offense Data

In [62]:
# open the alias file and remove whitespace characters like `\n` at the end of each line 
with open("ODOC_Raw/Vendor_Offense_Extract_Text.dat") as f:
    content = f.readlines()
content = [x.strip() for x in content] 

# convert to npy array
off_arr = np.array(content)

# convert to df
off_df = pd.DataFrame(off_arr)

# rename the 1st column
off_df.rename(columns={ off_df.columns[0]: "Raw" }, inplace = True)

# PARASE OUT COLUMNS
off_df['STATUTE_CODE'] = off_df['Raw'].str.slice(stop=38)
off_df['Raw'] = off_df['Raw'].str.slice(start=38)
off_df['DESCRIPTION'] = off_df['Raw'].str.slice(stop=40)
off_df['Raw'] = off_df['Raw'].str.slice(start=40)
off_df['VIOLENT'] = off_df['Raw'].str.slice(stop=1)
off_df['Raw'] = off_df['Raw'].str.slice(start=1)

# drop the original 'raw' column
del off_df['Raw']
off_df

# write to compressed pickle
compressed_pickle('offense_compressed.pickle', off_df.to_pickle('offense.pickle'))

off_df

,STATUTE_CODE,DESCRIPTION,VIOLENT
0,0-0,UNKNOWN - FOR WARRANTS ONLY,N
1,10-1144,ACTS CAUSING JUVENILE DELINQUENCY,N
2,10-1627,DEPRIVATION OF LAWFUL CUSTODY,N
3,10-26,UNLAWFUL ASSUMPTION OF CUSTODY OF CHILD,N
4,10-404.1,SEX OFFENDER PROVIDING SERVICES TO CHILD,N
...,...,...,...
1171,74-924,FILE FALSE STATEMENT PUBLIC EMPLOYEE RET,N
1172,76-23,TAMPERING WITH UTILITIES,N
1173,82-1020.22,VIOLATION OF WATER RESOURCES RULES/REGS,N
1174,82-105.2,UNAUTHORIZED USE OF WATER,N


### Profile Data

In [8]:
# open the alias file and remove whitespace characters like `\n` at the end of each line 
with open("ODOC_Raw/Vendor_Profile_Extract_Text.dat") as f:
    content = f.readlines()
content = [x.strip() for x in content] 

# convert to npy array
prof_arr = np.array(content)

# convert to df
prof_df = pd.DataFrame(prof_arr)

# rename the 1st column
prof_df.rename(columns={ prof_df.columns[0]: "Raw" }, inplace = True)

# PARSE OUT COLUMNS
prof_df['DOC_NUM'] = prof_df['Raw'].str.slice(stop=10)
prof_df['Raw'] = prof_df['Raw'].str.slice(start=10)
prof_df['LAST_NAME'] = prof_df['Raw'].str.slice(stop=30)
prof_df['Raw'] = prof_df['Raw'].str.slice(start=30)
prof_df['FIRST_NAME'] = prof_df['Raw'].str.slice(stop=30)
prof_df['Raw'] = prof_df['Raw'].str.slice(start=30)
prof_df['MIDDLE_NAME'] = prof_df['Raw'].str.slice(stop=30)
prof_df['Raw'] = prof_df['Raw'].str.slice(start=30)
prof_df['SUFFIX'] = prof_df['Raw'].str.slice(stop=5)
prof_df['Raw'] = prof_df['Raw'].str.slice(start=5)
prof_df['LAST_MOVE_DATE'] = prof_df['Raw'].str.slice(stop=9)
prof_df['Raw'] = prof_df['Raw'].str.slice(start=9)
prof_df['FACILITY'] = prof_df['Raw'].str.slice(stop=40)
prof_df['Raw'] = prof_df['Raw'].str.slice(start=40)
prof_df['BIRTH_DATE'] = prof_df['Raw'].str.slice(stop=9)
prof_df['Raw'] = prof_df['Raw'].str.slice(start=9)
prof_df['SEX'] = prof_df['Raw'].str.slice(stop=1)
prof_df['Raw'] = prof_df['Raw'].str.slice(start=1)
prof_df['RACE'] = prof_df['Raw'].str.slice(stop=40)
prof_df['Raw'] = prof_df['Raw'].str.slice(start=40)
prof_df['HAIR'] = prof_df['Raw'].str.slice(stop=40)
prof_df['Raw'] = prof_df['Raw'].str.slice(start=40)
prof_df['HEIGHT_FT'] = prof_df['Raw'].str.slice(stop=2)
prof_df['Raw'] = prof_df['Raw'].str.slice(start=2)
prof_df['HEIGHT_IN'] = prof_df['Raw'].str.slice(stop=2)
prof_df['Raw'] = prof_df['Raw'].str.slice(start=2)
prof_df['WEIGHT'] = prof_df['Raw'].str.slice(stop=4)
prof_df['Raw'] = prof_df['Raw'].str.slice(start=4)
prof_df['EYE'] = prof_df['Raw'].str.slice(stop=40)
prof_df['Raw'] = prof_df['Raw'].str.slice(start=40)
prof_df['STATUS'] = prof_df['Raw'].str.slice(stop=10)
prof_df['Raw'] = prof_df['Raw'].str.slice(start=10)

# drop the original 'raw' column
del prof_df['Raw']
prof_df

# write to compressed pickle
compressed_pickle('profile_compressed.pickle', prof_df.to_pickle('profile.pickle'))

prof_df

,DOC_NUM,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,LAST_MOVE_DATE,FACILITY,BIRTH_DATE,SEX,RACE,HAIR,HEIGHT_FT,HEIGHT_IN,WEIGHT,EYE,STATUS
0,0000008359,MESLES,ARDELL,,,03-MAR-86,INACTIVE,06-APR-87,M,,,,,,,Inactive
1,0000010337,SAWYER,FRANK,N,,12-JUN-91,INACTIVE,01-JAN-00,M,WHITE,UNK,5,1,,BLACK,Inactive
2,0000012556,GOFORTH,PAMELA,B,,13-NOV-89,INACTIVE,06-FEB-51,M,NATIVE AMERICAN,BROWN,5,9,116,N/R,Inactive
3,0000020413,KNIGHTON,RAYMOND,J,,18-APR-80,INACTIVE,20-AUG-48,M,BLACK,BROWN,,,,BROWN,Inactive
4,0000022574,STOCKTON,MELVIN,H,,02-JAN-97,INACTIVE,07-DEC-45,M,WHITE,BROWN,5,10,150,GRAY,Inactive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320979,0000990510,SKAGGS,BOBBY,L,,01-AUG-94,INACTIVE,26-JAN-33,M,WHITE,BROWN,5,10,145,HAZEL,Inactive
320980,0000993597,FOSTER,LAFONDA,F,,07-AUG-01,INACTIVE,30-AUG-63,F,WHITE,RED,5,7,170,BLUE,Inactive
320981,0000993890,GUINN,JEANETTE,,,15-SEP-94,INACTIVE,09-APR-39,F,WHITE,BROWN,5,4,118,BROWN,Inactive
320982,0000996281,PILON,GARY,V,,19-OCT-94,INACTIVE,18-SEP-53,M,WHITE,BROWN,6,2,185,GREEN,Inactive


### Sentence Data

In [98]:
# open the alias file and remove whitespace characters like `\n` at the end of each line 
with open("ODOC_Raw/Vendor_sentence_Extract_Text.dat") as f:
    content = f.readlines()
content = [x.strip() for x in content] 

# convert to npy array
sent_arr = np.array(content)

# convert to df
sent_df = pd.DataFrame(sent_arr)

# rename the 1st column
sent_df.rename(columns={ sent_df.columns[0]: "Raw" }, inplace = True)

# PARSE OUT COLUMNS
sent_df['DOC_NUM'] = sent_df['Raw'].str.slice(stop=10)
sent_df['Raw'] = sent_df['Raw'].str.slice(start=10)
sent_df['STATUTE_CODE'] = sent_df['Raw'].str.slice(stop=40)
sent_df['Raw'] = sent_df['Raw'].str.slice(start=40)
sent_df['SENTENCING_COUNTY'] = sent_df['Raw'].str.slice(stop=40)
sent_df['Raw'] = sent_df['Raw'].str.slice(start=40)
sent_df['JS_DATE'] = sent_df['Raw'].str.slice(stop=8)
sent_df['Raw'] = sent_df['Raw'].str.slice(start=8)
sent_df['CRF_NUMBER'] = sent_df['Raw'].str.slice(stop=40)
sent_df['Raw'] = sent_df['Raw'].str.slice(start=40)
sent_df['INCARCERATED_TERM_IN_YEARS'] = sent_df['Raw'].str.slice(stop=10)
sent_df['Raw'] = sent_df['Raw'].str.slice(start=10)
sent_df['PROBATION_TERM_IN_YEARS'] = sent_df['Raw'].str.slice(stop=10)
sent_df['Raw'] = sent_df['Raw'].str.slice(start=10)

# remove white space in numeric columns
sent_df['INCARCERATED_TERM_IN_YEARS'] = sent_df['INCARCERATED_TERM_IN_YEARS'].str.strip()
sent_df['PROBATION_TERM_IN_YEARS'] = sent_df['PROBATION_TERM_IN_YEARS'].str.strip()

# SET COLUMN TYPES
#sent_df["DOC_NUM"] = pd.to_numeric(sent_df["DOC_NUM"])
sent_df['INCARCERATED_TERM_IN_YEARS'] = pd.to_numeric(sent_df['INCARCERATED_TERM_IN_YEARS'])
#sent_df['PROBATION_TERM_IN_YEARS'] = pd.to_numeric(sent_df['PROBATION_TERM_IN_YEARS'])

# drop the original 'raw' column
del sent_df['Raw']
sent_df

# filter to Tulsa County Only
sent_df = sent_df[sent_df["SENTENCING_COUNTY"].str.contains("TULSA COUNTY COURT")]

# Write to compressed pickle
compressed_pickle('sentence_compressed.pickle', sent_df.to_pickle('sentence.pickle'))

sent_df

,DOC_NUM,STATUTE_CODE,SENTENCING_COUNTY,JS_DATE,CRF_NUMBER,INCARCERATED_TERM_IN_YEARS,PROBATION_TERM_IN_YEARS
3,0000020413,21-1731,TULSA COUNTY COURT,19-APR-7,777-240,NaN,00000
4,0000022574,21-741,TULSA COUNTY COURT,03-MAR-6,420-393,777.0,7.00
5,0000022574,47-11-902,TULSA COUNTY COURT,24-OCT-9,494-4167,NaN,00000
6,0000022663,21-801.A,TULSA COUNTY COURT,22-APR-6,4 2-513,3.0,6.00
7,0000022663,21-801.A,TULSA COUNTY COURT,22-APR-6,4 2-513,1.0,2.00
...,...,...,...,...,...,...,...
1359998,0000864451,21-1283,TULSA COUNTY COURT,05-OCT-2,0CF-2020-88,NaN,00000
1359999,0000864461,21-1550.41,TULSA COUNTY COURT,31-JUL-2,02020-1486,0.0,1.00
1360000,0000864461,21-1550.41,TULSA COUNTY COURT,31-JUL-2,02020-1486,NaN,00000
1360088,0000864667,21-1720,TULSA COUNTY COURT,07-DEC-2,020-2701,NaN,


### Combine all the dataframes into one master df - Inner Joins on DOC_NUM & STATUTE_CODE

In [107]:
# set to view all columns for QC
pd.set_option('display.max_columns', None)

# Start with Sentence and Profile on DocNum using innjer join
sent_prof_MERGE = pd.merge(left=sent_df, right=prof_df, left_on='DOC_NUM', right_on='DOC_NUM')

# Bring in Offense based on DocNum using inner join
sent_prof_MERGE['STATUTE_CODE'] = sent_prof_MERGE['STATUTE_CODE'].str.strip()
off_df['STATUTE_CODE'] = off_df['STATUTE_CODE'].str.strip()
off_prof_sent_MERGE = pd.merge(left=sent_prof_MERGE, right=off_df, left_on='STATUTE_CODE', right_on='STATUTE_CODE')

# drop duplicate rows
odoc_final = off_prof_sent_MERGE
# odoc_final = odoc_merge.drop_duplicates(keep = 'first')

# View final dataframe before edits
odoc_final

,DOC_NUM,STATUTE_CODE,SENTENCING_COUNTY,JS_DATE,CRF_NUMBER,INCARCERATED_TERM_IN_YEARS,PROBATION_TERM_IN_YEARS,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,LAST_MOVE_DATE,FACILITY,BIRTH_DATE,SEX,RACE,HAIR,HEIGHT_FT,HEIGHT_IN,WEIGHT,EYE,STATUS,DESCRIPTION,VIOLENT
0,0000020413,21-1731,TULSA COUNTY COURT,19-APR-7,777-240,NaN,00000,KNIGHTON,RAYMOND,J,,18-APR-80,INACTIVE,20-AUG-48,M,BLACK,BROWN,,,,BROWN,Inactive,LARCENY OF MERCH FROM RETAILER,N
1,0000071854,21-1731,TULSA COUNTY COURT,18-NOV-9,191-4524,0.0,3.00,WASHINGTON,DONALD,R,SR,13-APR-93,INACTIVE,12-JAN-46,M,BLACK,BLACK,6,3,220,BROWN,Inactive,LARCENY OF MERCH FROM RETAILER,N
2,0000074305,21-1731,TULSA COUNTY COURT,19-NOV-9,393-3374,0.0,2.00,GATEWOOD,HERMAN,,,01-SEP-94,INACTIVE,12-MAR-43,M,BLACK,BLACK,5,7,165,BROWN,Inactive,LARCENY OF MERCH FROM RETAILER,N
3,0000076538,21-1731,TULSA COUNTY COURT,17-MAR-8,080-438,0.0,2.00,BLAYLOCK,ROBERT,C,,16-MAR-82,INACTIVE,23-JUL-38,M,WHITE,BLONDE,6,1,220,N/R,Inactive,LARCENY OF MERCH FROM RETAILER,N
4,0000076709,21-1731,TULSA COUNTY COURT,23-AUG-9,393-3429,0.0,2.00,MACK,HERMAN,E,,06-NOV-08,INACTIVE,23-JUN-45,M,BLACK,"GRAY, ETC.",5,6,240,BROWN,Inactive,LARCENY OF MERCH FROM RETAILER,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209618,0000854693,47-2-129,TULSA COUNTY COURT,14-JAN-2,0CF-19-4659,NaN,00000,JOHNSON,JOSHUA,DEAN,,,NORTHEAST DISTRICT OFFICE,14-JUN-91,M,WHITE,BLONDE,6,3,200,BLUE,Active,DISSEMINATING CONFIDENTIAL INFORMATION,N
209619,0000854693,47-2-129,TULSA COUNTY COURT,20-JUL-2,0CF-19-4659,NaN,00000,JOHNSON,JOSHUA,DEAN,,,NORTHEAST DISTRICT OFFICE,14-JUN-91,M,WHITE,BLONDE,6,3,200,BLUE,Active,DISSEMINATING CONFIDENTIAL INFORMATION,N
209620,0000857822,69-1211,TULSA COUNTY COURT,05-JUN-2,0CF-2020-1845,NaN,00000,RUVALCABA,GILBERTO,MANUEL,,,TULSA COUNTY DISTRICT,04-FEB-90,M,HISPANIC,,,,,,Active,CONSTRUCTION OF STATE/FED RIGHT-OF-WAY,N
209621,0000858022,2-6-125,TULSA COUNTY COURT,15-JUN-2,0CF-2019-1661,NaN,00000,DIOSDADO,DAVID,,,,TULSA COUNTY DISTRICT,10-JAN-92,M,HISPANIC,,,,,,Active,MOVING QUARANTINED LIVESTOCK,N


### Compress the Pickles for Git Storage Requirements

### Decompress Pickles

In [12]:
# decompress files
sentence = decompress_pickle('sentence_compressed.pickle.pbz2')
alias = decompress_pickle('alias_compressed.pickle.pbz2')
offense = decompress_pickle('offense_compressed.pickle.pbz2')
profile = decompress_pickle('profile_compressed.pickle.pbz2')

In [13]:
# View one for sanity check
sentence = pd.read_pickle('sentence.pickle')
sentence

,DOC_NUM,STATUTE_CODE,SENTENCING_COUNTY,JS_DATE,CRF_NUMBER,INCARCERATED_TERM_IN_YEARS,PROBATION_TERM_IN_YEARS
0,8359,21-1720,POTTAWATOMIE COUNTY COURT,13-MAR-1,32010-300,NaN,00000
1,10337,21-701.7,BRYAN COUNTY COURT,13-APR-2,0 1-0,777.0,7.00
2,10337,21-701.7,BRYAN COUNTY COURT,14-DEC-6,220-15645,NaN,00000
3,20413,21-1731,TULSA COUNTY COURT,19-APR-7,777-240,NaN,00000
4,22574,21-741,TULSA COUNTY COURT,03-MAR-6,420-393,777.0,7.00
...,...,...,...,...,...,...,...
1360620,996281,21-798,UNKNOWN COUNTY COURT,08-JUN-8,2 9-897,777.0,7.00
1360621,996281,21-444,WASHINGTON JURISDICTION,29-MAY-8,079-1236,1.0,0.00
1360622,997494,21-799,CALIFORNIA JURISDICTION,01-FEB-9,494-8235,0.0,5.00
1360623,997494,21-799,CALIFORNIA JURISDICTION,01-FEB-9,494-8235,0.0,3.00
